In [1]:
import sys
import random
import os
from pathlib import Path
import shutil
import json

import argparse
import tqdm
import spacy
from spacy.gold import minibatch
from spacy.language import Language
from spacy import util

In [2]:
from scispacy.data_util import read_full_med_mentions, read_ner_from_tsv
from scispacy.per_class_scorer import PerClassScorer
from scispacy.train_utils import evaluate_ner

In [3]:
def train_ner(output_dir: str,
              train_data_path: str,
              dev_data_path: str,
              test_data_path: str,
              run_test: bool = None,
              model: str = None,
              n_iter: int = 10,
              meta_overrides: str = None):

    util.fix_random_seed(util.env_opt("seed", 0))
    train_data = read_ner_from_tsv(train_data_path)
    dev_data = read_ner_from_tsv(dev_data_path)
    test_data = read_ner_from_tsv(test_data_path)
    os.makedirs(output_dir, exist_ok=True)
    if run_test:
        nlp = spacy.load(model)
        print("Loaded model '%s'" % model)
        evaluate_ner(nlp, dev_data, dump_path=os.path.join(output_dir, "dev_metrics.json"))
        evaluate_ner(nlp, test_data, dump_path=os.path.join(output_dir, "test_metrics.json"))
    else:
        train(model, train_data, dev_data, test_data, output_dir, n_iter, meta_overrides)

In [4]:
def train(model, train_data, dev_data, test_data, output_dir, n_iter, meta_overrides):
    """Load the model, set up the pipeline and train the entity recognizer."""
    
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    if meta_overrides is not None:
        metadata = json.load(open(meta_overrides))
        nlp.meta.update(metadata)

    original_tokenizer = nlp.tokenizer

#     nlp.tokenizer = nlp_en.tokenizer

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names and "parser" in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, after="parser")
    elif 'ner' not in nlp.pipe_names and "tagger" in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, after="tagger")
    elif 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in train_data:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    dropout_rates = util.decaying(util.env_opt('dropout_from', 0.2),
                                  util.env_opt('dropout_to', 0.2),
                                  util.env_opt('dropout_decay', 0.005))
    batch_sizes = util.compounding(util.env_opt('batch_from', 1),
                                   util.env_opt('batch_to', 32),
                                   util.env_opt('batch_compound', 1.001))

    optimizer = nlp.begin_training()
    best_epoch = 0
    best_f1 = 0
    for i in range(n_iter):
        print(str(i)+'--'+str(n_iter))
        random.shuffle(train_data)
        count = 0
        losses = {}
        total = len(train_data)

        with nlp.disable_pipes(*other_pipes):  # only train NER
            with tqdm.tqdm(total=total, leave=True) as pbar:
                for batch in minibatch(train_data, size=batch_sizes):
                    docs, golds = zip(*batch)
                    nlp.update(docs, golds, sgd=optimizer,
                               losses=losses, drop=next(dropout_rates))
                    pbar.update(len(batch))
                    if count % 100 == 0 and count > 0:
                        print('sum loss: %s' % losses['ner'])
                    count += 1

        # save model to output directory
        output_dir_path = Path(output_dir + "/" + str(i))
        if not output_dir_path.exists():
            output_dir_path.mkdir()

        with nlp.use_params(optimizer.averages):
            nlp.tokenizer = original_tokenizer
            nlp.to_disk(output_dir_path)
            print("Saved model to", output_dir_path)

        # test the saved model
        print("Loading from", output_dir_path)
        nlp2 = util.load_model_from_path(output_dir_path)
#         nlp2.tokenizer = nlp_en.tokenizer

        metrics = evaluate_ner(nlp2, dev_data)
        if metrics["f1-measure-overall"] > best_f1:
            best_f1 = metrics["f1-measure-overall"]
            best_epoch = i
    # save model to output directory
    best_model_path = Path(output_dir + "/" + "best")
    print(f"Best Epoch: {best_epoch} of {n_iter}")
    if os.path.exists(best_model_path):
        shutil.rmtree(best_model_path)
    shutil.copytree(os.path.join(output_dir, str(best_epoch)),
                    best_model_path)

    # test the saved model
    print("Loading from", best_model_path)
    nlp2 = util.load_model_from_path(best_model_path)
#     nlp2.tokenizer = nlp_en.tokenizer

    evaluate_ner(nlp2, dev_data, dump_path=os.path.join(output_dir, "dev_metrics.json"))
    evaluate_ner(nlp2, test_data, dump_path=os.path.join(output_dir, "test_metrics.json"))

In [5]:
model_output_dir = '/home/santosh/Work/models/SpaCy/en-cord19-lg/'
train_data_path = '/home/santosh/Work/GitHub/COVID-19-Named-Entity-Recognition-/Datasets/BIO/non_overlapping_dataset/train.csv'
dev_data_path = '/home/santosh/Work/GitHub/COVID-19-Named-Entity-Recognition-/Datasets/BIO/non_overlapping_dataset/devel.csv'
test_data_path = '/home/santosh/Work/GitHub/COVID-19-Named-Entity-Recognition-/Datasets/BIO/non_overlapping_dataset/test.csv'
run_test = False
model_path = '/home/santosh/Work/models/SpaCy/cord19.spacy_word2vec.model/' # None #'en_core_sci_md'
iterations = 20
meta_overrides = '/home/santosh/Work/models/SpaCy/cord19.spacy_word2vec.model/meta.json'

In [6]:
train_ner(model_output_dir,
              train_data_path,
              dev_data_path,
              test_data_path,
              run_test,
              model_path,
              iterations,
              meta_overrides)

Loaded model '/home/santosh/Work/models/SpaCy/cord19.spacy_word2vec.model/'


  0%|          | 3/105736 [00:00<1:11:44, 24.56it/s]

0--20


  0%|          | 107/105736 [00:03<54:25, 32.35it/s] 

sum loss: 814.178682599605


...

precision-CHED: 0.9003176523975193
recall-CHED: 0.9289839238333073
f1-measure-CHED: 0.9144261791365301
precision-COMP: 0.8953488372093024
recall-COMP: 0.9482758620689655
f1-measure-COMP: 0.9210526315788975
precision-FUNC: 0.8924387646432373
recall-FUNC: 0.9426321709786275
f1-measure-FUNC: 0.9168490153172366
precision-ANAT: 0.9168097187376205
recall-ANAT: 0.9257333333333333
f1-measure-ANAT: 0.9212499170702082
precision-PATH: 0.8875219683655534
recall-PATH: 0.9215328467153283
f1-measure-PATH: 0.9042076991942203
precision-DISO: 0.9035689081560642
recall-DISO: 0.9403032051618319
f1-measure-DISO: 0.9215701395850507
precision-ENZY: 0.8703071672354945
recall-ENZY: 0.8793103448275859
f1-measure-ENZY: 0.8747855917666736
precision-PRGE: 0.7602868362642992
recall-PRGE: 0.7726878361964254
f1-measure-PRGE: 0.7664371772805008
precision-PROC: 0.9172595112898237
recall-PROC: 0.9365229748934154
f1-measure-PROC: 0.9267911555589781
precision-SPEC: 0.9215964663196088
recall-SPEC: 0.952318852392208
f1-meas

 97%|█████████▋| 22061/22658 [01:01<00:01, 358.15it/s]

precision-CHED: 0.9012058695336336
recall-CHED: 0.9284538242778028
f1-measure-CHED: 0.9146269537009233
precision-COMP: 0.8954010095344924
recall-COMP: 0.9488855869242199
f1-measure-COMP: 0.9213677679987958
precision-FUNC: 0.8937691521961184
recall-FUNC: 0.9418729817007534
f1-measure-FUNC: 0.9171907756812917
precision-ANAT: 0.9170156071564523
recall-ANAT: 0.926657263751763
f1-measure-ANAT: 0.9218112244897461
precision-PATH: 0.8886986301369861
recall-PATH: 0.9202127659574466
f1-measure-PATH: 0.9041811846689394
precision-DISO: 0.9034559213380248
recall-DISO: 0.9398786541643684
f1-measure-DISO: 0.9213074465470131
precision-ENZY: 0.87378640776699
recall-ENZY: 0.8823529411764702
f1-measure-ENZY: 0.8780487804877547
precision-PRGE: 0.7596122247781795
recall-PRGE: 0.7721730415901119
f1-measure-PRGE: 0.765841133106883
precision-PROC: 0.9172362555720653
recall-PROC: 0.9362960715910814
f1-measure-PROC: 0.9266681678300183
precision-SPEC: 0.9213247414955792
recall-SPEC: 0.951776453285858
f1-measure-

100%|██████████| 22658/22658 [01:02<00:00, 360.19it/s]


precision-untyped: 		 0.9077198029728434
recall-untyped: 		 0.9360783961161557
f1-measure-untyped: 		 0.921681014451009
precision-overall: 		 0.8851874360286591
recall-overall: 		 0.9157222507155501
f1-measure-overall: 		 0.9001959811662412


In [9]:
## Test best model performance on test set
best_model_path = model_output_dir+'best/'

print("Loading from", best_model_path)
nlp2 = util.load_model_from_path(best_model_path)

Loading from /home/santosh/Work/models/SpaCy/en-cord19-lg/best/


In [11]:
from spacy import displacy

In [30]:
# create distinct colours for labels
col_dict = {}
seven_colours = ['#00FFFF', '#B0C4DE', '#ADFF2F', '#FFA07A', '#E6E6FA', '#FFC0CB', '#FFFF00', '#BC8F8F', '#7FFFD4', '#D8BFD8']
for label, colour in zip(nlp2.pipe_labels['ner'], seven_colours):
    col_dict[label] = colour

options = {'ents': nlp2.pipe_labels['ner'], 'colors':col_dict}


spacy.displacy.render(sentence, style='ent', jupyter=True, options=options)

In [6]:
# Package the Best model

!python -m spacy package /home/santosh/Work/models/SpaCy/en-cord19-lg/best/ /home/santosh/Work/models/SpaCy/en_covido


✔ Loaded meta.json from file
/home/santosh/Work/models/SpaCy/en-cord19-lg/best/meta.json
✔ Successfully created package 'en_covido-0.0.0'
/home/santosh/Work/models/SpaCy/en_covido/en_covido-0.0.0
To build the package, run `python setup.py sdist` in this directory.
